In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
file = open('test files/detected_results.txt','r')
all_lines = file.readlines()

df_columns = all_lines[0].split('\t')
df_columns[-1] = df_columns[-1][:-1]

df = pd.DataFrame(columns = df_columns)

for line in tqdm(all_lines[1:]):
    split_line = line.split('\t')
    split_line[-1] = split_line[-1][:-1]
    line_dict = dict(zip(df_columns,split_line))
    df = df.append(line_dict, ignore_index=True)
    
file.close()

100%|██████████████████████████████████████████████████████████████████████████████| 6993/6993 [01:33<00:00, 75.12it/s]


In [3]:
df = df[df.columns[5:]].copy()
for column in df.columns:
    df[column] = df[column].apply(lambda x : float(x))

In [4]:
#for non-square image, would need to calculate x and y as separate image size and dimension

image_size = 2845 #number of pixels in one image dimension
image_dim = 1181.05 #length of image dimension

#calculate cell centroid coordinates as pixel
df['Centroid X pixel'] = df['Centroid X Âµm']*image_size/image_dim
df['Centroid Y pixel'] = df['Centroid Y Âµm']*image_size/image_dim

df['Centroid X pixel'] = df['Centroid X pixel'].apply(lambda x:int(x))
df['Centroid Y pixel'] = df['Centroid Y pixel'].apply(lambda x:int(x))

In [5]:
#drop extra columns and rename channel columns

df = df[['Centroid X pixel',
        'Centroid Y pixel',
        'Nucleus: Ch3-T1 mean',#nucleus
        'Nucleus: Ch2 GaAsP-T3 mean',#red
        'Nucleus: Ch3-T2 mean',#green
        'Nucleus: Ch1-T4 mean',#blue
        'Nucleus: Area',
        'Nucleus: Circularity']].copy()

df = df.rename(columns = {'Nucleus: Ch3-T1 mean':'channel_nucleus',
                            'Nucleus: Ch3-T2 mean':'channel_green',
                            'Nucleus: Ch1-T4 mean':'channel_blue',
                            'Nucleus: Ch2 GaAsP-T3 mean':'channel_red'})

In [6]:
#add sum of channels column
df['channel_sum'] = df['channel_red']+df['channel_green']+df['channel_blue']

In [7]:
#add columns of channel value as fraction of total
#all cells to be calculated as ratios

df['ratio_red'] = (df['channel_red']/df['channel_sum'])
df['ratio_green'] = (df['channel_green']/df['channel_sum'])
df['ratio_blue'] = (df['channel_blue']/df['channel_sum'])

In [8]:
#add channel RGB image value as ratio of sum of all channels
#e.g. R=1,G=0,B=0 ratio would be rgb red = 255,rgb green = 0,rgb blue = 0
#and  R=1,G=1,B=1 ratio would be rgb red = 85,rgb green = 85,rgb blue = 85

df['rgb_red'] = (df['channel_red']/df['channel_sum'])*255
df['rgb_green'] = (df['channel_green']/df['channel_sum'])*255
df['rgb_blue'] = (df['channel_blue']/df['channel_sum'])*255

In [9]:
df = df.rename(columns={'Centroid X pixel':'centroid_x','Centroid Y pixel':'centroid_y','Nucleus: Area':'nucleus_area','Nucleus: Circularity':'nucleus_circularity'})

In [10]:
df = df[['centroid_x','centroid_y','channel_nucleus','nucleus_area','nucleus_circularity',
              'channel_red','channel_green','channel_blue','channel_sum','ratio_red','ratio_green',
              'ratio_blue','rgb_red','rgb_green','rgb_blue']].copy()

In [11]:
df.to_csv('test files/detected_results.csv',index=False)